## Building a Retrieval-Augmented Generation (RAG) System with LangChain

### Introduction

In this notebook, we will learn how to build a Retrieval-Augmented Generation (RAG) system using LangChain in Python. RAG systems combine information retrieval and natural language generation to produce answers that are grounded in external knowledge bases. This approach is particularly useful when dealing with large documents or datasets where direct querying isn’t efficient or possible.

### Objectives

- Understand the concept of Retrieval-Augmented Generation (RAG).
- Learn how to use LangChain to implement a RAG system.
- Implement the system step by step with guided TODO tasks.
- Test your implementation at each step.
- Provide helpful explanations and definitions.

Help

### Methods Used:

- LangChain: A library for building language model applications.
- VectorStore (FAISS): A tool for efficient similarity search and clustering of dense vectors.
- OpenAI Embeddings: Representations of text that can capture semantic meaning.
- RetrievalQA Chain: Combines retrieval and question-answering over documents.

### Data Used

- I extracted some chapters of the Gen AI course as a txt file. 
- The goal how this notebook is to build a RAG system that can answer questions based on the content of these chapters.

## Step 1: Set Up Your Environment

We need to import the required modules and set up the OpenAI API key.

In [1]:
# Import necessary libraries
import sys
from dotenv import load_dotenv

# from langchain import OpenAI, hub
# from langchain_openai import OpenAIEmbeddings
# from langchain_openai.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List

# import os

In [2]:
load_dotenv()
sys.path.append("../")

## Step 2: Load and Split Documents

Load the document you want to use and split it into manageable chunks.

In [3]:
# TODO: Load your document and split it into chunks
# Hint: Use TextLoader and RecursiveCharacterTextSplitter

filename = "../data/gen_ai_course.txt"
# Answer:
loader = TextLoader(filename)
documents = loader.load()

# Answer
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

## Step 3: Create Embeddings and Build the VectorStore

Generate embeddings for each chunk and store them in a vector store for efficient retrieval.

In [4]:
# TODO: Create embeddings and store them in a VectorStore
# Hint: Use GoogleGenerativeAIEmbeddings and FAISS

from langchain_google_genai import GoogleGenerativeAIEmbeddings

# from langchain_community.docstore.in_memory import InMemoryDocstore

# import faiss

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

## Step 4: Set Up the QA Chain using LCEL 

Create a chain that can retrieve relevant chunks and generate answers based on them.

In [5]:
# TODO: Create a RetrievalQA chain
# Hint: Use ChatOpenAI, create a prompt, and use StrOutputParser
# Hint: The chain should be an LCEL chain https://python.langchain.com/v0.1/docs/expression_language/get_started/

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a question answering chatbot. 
            You'll say if you don't know. 
            You'll find the relevant information in {formatted_docs}. 
            Answer in at most 5 sentences.""",
        ),
        ("human", "{query}"),
    ]
)


def format_docs(docs: List[Document]):
    return "\n\n".join(doc.page_content for doc in docs)


question = "What's a Retrieval Augmented Generation?"


formatted_docs = format_docs(docs)

# Answer:
qa_chain = prompt | llm
answer = qa_chain.invoke(
    {
        "input_language": "English" or "French",
        "output_language": "English",
        "query": question,
        "formatted_docs": formatted_docs,
    }
).content

print(f"Question: {question}\nAnswer: {answer}")

Question: What's a Retrieval Augmented Generation?
Answer: Retrieval-Augmented Generation (RAG) combines retrieval-based and generation-based models.  It enhances language models by giving them access to external knowledge or documents during generation. This allows the model to generate more accurate, up-to-date information by retrieving relevant data, rather than relying solely on its internal parameters, which reduces hallucinations.  RAG models are better at handling queries about recent events or specific information not present in their training data.



## Step 5: Ask Questions and Get Answers

Test the system by asking a question.

In [6]:
# TODO: Ask a question to the QA chain
# Replace 'Your question here' with an actual question and run the qa_chain for this question

# Answer:
query = "What's a Transformer?"
result = qa_chain.invoke(
    {
        "query": query,
        "formatted_docs": formatted_docs,
    }
).content
print(f"Question: {query}\nAnswer: {result}")

Question: What's a Transformer?
Answer: Transformers are a type of neural network architecture primarily used in natural language processing.  They rely on a mechanism called self-attention, which allows the model to weigh the importance of different parts of the input data when processing it. Unlike recurrent neural networks, Transformers process data in parallel, making them significantly faster to train.  They have become the dominant architecture for large language models due to their ability to handle long-range dependencies and capture complex relationships in text.



## Step 6: Test Your Implementation with Different Questions

Try out different questions to see how the system performs.

In [7]:
# Replace 'Another question here' with your own question and run the qa_chain for this question

query = "Can you summarize the key points mentioned?"
result = qa_chain.invoke(
    {
        "query": query,
        "formatted_docs": formatted_docs,
    }
)
print(result.content)

Supervised Fine-Tuning (SFT) trains LLMs on human-written answers to follow instructions but is limited by data cost and potential hallucinations.  Reinforcement Learning from Human Feedback (RLHF) uses a reward model trained on human preferences to improve alignment, but faces challenges like answer length inflation and human disagreement.  Direct Preference Optimization (DPO) simplifies RLHF by directly optimizing for human preferences.  Retrieval Augmented Generation (RAG) enhances LLMs by retrieving relevant information from external sources, improving factuality and reducing hallucinations.  Evaluation of RAG models is complex, often relying on LLM-based judges or metrics like precision and recall.



## Step 7: Improve the System

You can experiment with different parameters, like adjusting the chunk size or using a different language model.

Conclusion

Congratulations! You’ve built a simple Retrieval-Augmented Generation system using LangChain. This system can retrieve relevant information from documents and generate answers to user queries.

Help

- TextLoader: Loads text data from files.
- RecursiveCharacterTextSplitter: Splits text into smaller chunks for better processing.
- FAISS: A library for efficient similarity search of embeddings.
- RetrievalQA Chain: A chain that retrieves relevant documents and answers questions based on them.
- OpenAIEmbeddings: Generates embeddings that capture the semantic meaning of text.

## Help

In [8]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

prompt_value = template.invoke({"name": "Bob", "user_input": "What is your name?"})

# Output:
# ChatPromptValue(
#    messages=[
#        SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
#        HumanMessage(content='Hello, how are you doing?'),
#        AIMessage(content="I'm doing well, thanks!"),
#        HumanMessage(content='What is your name?')
#    ]
# )